# IMP Parsing

Definition of the CFG in the Lark-style EBNF.

In [2]:
import outlines

In [7]:
imp_grammar = r"""
    ########################
    #  Top-level & program #
    ########################
    ?start: command                       -> program        # one or more commands

    ################
    #  Commands    #
    ################
    ?command: simple
            | simple ";" command           -> seq            # right-associative sequencing

    ?simple: "skip"                        -> skip
           | NAME ":=" aexpr               -> assign
           | "if" bexpr "then" command "else" command                -> if
           | "while" bexpr "do" command    -> while
           | "(" command ")"                               # optional grouping

    ########################
    #  Arithmetic grammar  #
    ########################
    ?aexpr: term
          | aexpr "+" term                 -> add
          | aexpr "-" term                 -> sub

    ?term: factor
         | term "*" factor                 -> mul
         | term "/" factor                 -> div

    ?factor: NUMBER                        -> number
           | NAME                          -> var
           | "-" factor                    -> neg
           | "(" aexpr ")"                 -> aparen

    #######################
    #  Boolean grammar    #
    #######################
    ?bexpr: bterm
          | bexpr "||" bterm               -> or

    ?bterm: bfactor
          | bterm "&&" bfactor             -> and

    ?bfactor: "true"                       -> true
            | "false"                      -> false
            | "!" bfactor                  -> not
            | aexpr relop aexpr            -> rel
            | "(" bexpr ")"                -> bparen

    relop: "==" | "!=" | "<" | "<=" | ">" | ">="

    #################
    #  Terminals    #
    #################
    NAME: /[A-Za-z_][A-Za-z0-9_]*/          # identifiers
    %import common.NUMBER                   # ints from Lark’s standard library
    %import common.WS_INLINE
    %import common.WS
    %ignore WS_INLINE                       # skip spaces and tabs
    %ignore WS
"""

In [4]:
model = outlines.models.transformers("Qwen/Qwen2.5-0.5B")
generator = outlines.generate.cfg(model, imp_grammar)
sequence = generator("In the programming language IMP, a code that do : x=10, y=0, while x>0: y = y + 2, x = x - 1, would be in IMP:", max_tokens=50)

print(sequence)

NameError: name 'imp_grammar' is not defined

x:=10; y:=0; while x>0 do y:=y+2; x:=x-1; endwhile:= y; What

No error: endwhile is just a variable name here, we assign the value in y to endwhile

# ConstraintLM

In [2]:
import constraintlm as clm
import torch

c:\Users\cleme\anaconda3\envs\llmsmc\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
qwenllm = clm.TransformersLM("Qwen/Qwen2.5-0.5B")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [6]:
prompt = ["In the programming language IMP, a code that do : x=10, y=0, while x>0: y = y + 2, x = x - 1, would be in IMP:"]
batch = qwenllm.tokenizer(prompt, padding=True, return_tensors="pt")

In [7]:
cfg = clm.CLMCFGLogitsProcessor(imp_grammar, model.tokenizer, qwenllm)

In [8]:
cfg_multinomial = clm.MultinomialSeqSampler(qwenllm, logits_processor=cfg)
cons_generated_token_ids = cfg_multinomial.sample(batch.input_ids, max_length=10, top_k=5)
print(qwenllm.tokenizer.batch_decode(torch.cat([batch.input_ids, cons_generated_token_ids], dim=-1)))

1
2
3
4
5
6
7
8
9
['In the programming language IMP, a code that do : x=10, y=0, while x>0: y = y + 2, x = x - 1, would be in IMP:  \t(x:= 10;y:=']


# Python parsing 

for := x; y:=y+2'

This code is valid because we are not checking that variable are assigned, and for is not a keyword here.

%import common.NEWLINE   -> _NL
%ignore _NL
%ignore /[ \t]+/         // whitespace

_NEWLINE: ( /\r?\n[\t ]*/ | COMMENT )+

%ignore /[\t \f]+/  // WS
%ignore /\\[\t \f]*\r?\n/   // LINE_CONT
%ignore COMMENT
%declare _INDENT _DEDENT

In [5]:
python_grammar = r"""

start: file_input

single_input: _NEWLINE | simple_stmt | compound_stmt _NEWLINE
file_input: (_NEWLINE | stmt)*
eval_input: testlist _NEWLINE*

decorator: "@" dotted_name [ "(" [arguments] ")" ] _NEWLINE
decorators: decorator+
decorated: decorators (classdef | funcdef | async_funcdef)

async_funcdef: "async" funcdef
funcdef: "def" name "(" [parameters] ")" ["->" test] ":" suite

parameters: paramvalue ("," paramvalue)* ["," SLASH ("," paramvalue)*] ["," [starparams | kwparams]]
          | starparams
          | kwparams

SLASH: "/" // Otherwise the it will completely disappear and it will be undisguisable in the result
starparams: (starparam | starguard) poststarparams
starparam: "*" typedparam
starguard: "*"
poststarparams: ("," paramvalue)* ["," kwparams]
kwparams: "**" typedparam ","?

?paramvalue: typedparam ("=" test)?
?typedparam: name (":" test)?


lambdef: "lambda" [lambda_params] ":" test
lambdef_nocond: "lambda" [lambda_params] ":" test_nocond
lambda_params: lambda_paramvalue ("," lambda_paramvalue)* ["," [lambda_starparams | lambda_kwparams]]
          | lambda_starparams
          | lambda_kwparams
?lambda_paramvalue: name ("=" test)?
lambda_starparams: "*" [name]  ("," lambda_paramvalue)* ["," [lambda_kwparams]]
lambda_kwparams: "**" name ","?


?stmt: simple_stmt | compound_stmt
?simple_stmt: small_stmt (";" small_stmt)* [";"] _NEWLINE
?small_stmt: (expr_stmt | assign_stmt | del_stmt | pass_stmt | flow_stmt | import_stmt | global_stmt | nonlocal_stmt | assert_stmt)
expr_stmt: testlist_star_expr
assign_stmt: annassign | augassign | assign

annassign: testlist_star_expr ":" test ["=" test]
assign: testlist_star_expr ("=" (yield_expr|testlist_star_expr))+
augassign: testlist_star_expr augassign_op (yield_expr|testlist)
!augassign_op: "+=" | "-=" | "*=" | "@=" | "/=" | "%=" | "&=" | "|=" | "^=" | "<<=" | ">>=" | "**=" | "//="
?testlist_star_expr: test_or_star_expr
                   | test_or_star_expr ("," test_or_star_expr)+ ","?  -> tuple
                   | test_or_star_expr ","  -> tuple

del_stmt: "del" exprlist
pass_stmt: "pass"
?flow_stmt: break_stmt | continue_stmt | return_stmt | raise_stmt | yield_stmt
break_stmt: "break"
continue_stmt: "continue"
return_stmt: "return" [testlist]
yield_stmt: yield_expr
raise_stmt: "raise" [test ["from" test]]
import_stmt: import_name | import_from
import_name: "import" dotted_as_names
import_from: "from" (dots? dotted_name | dots) "import" ("*" | "(" import_as_names ")" | import_as_names)
!dots: "."+
import_as_name: name ["as" name]
dotted_as_name: dotted_name ["as" name]
import_as_names: import_as_name ("," import_as_name)* [","]
dotted_as_names: dotted_as_name ("," dotted_as_name)*
dotted_name: name ("." name)*
global_stmt: "global" name ("," name)*
nonlocal_stmt: "nonlocal" name ("," name)*
assert_stmt: "assert" test ["," test]

?compound_stmt: if_stmt | while_stmt | for_stmt | try_stmt | match_stmt
              | with_stmt | funcdef | classdef | decorated | async_stmt
async_stmt: "async" (funcdef | with_stmt | for_stmt)
if_stmt: "if" test ":" suite elifs ["else" ":" suite]
elifs: elif_*
elif_: "elif" test ":" suite
while_stmt: "while" test ":" suite ["else" ":" suite]
for_stmt: "for" exprlist "in" testlist ":" suite ["else" ":" suite]
try_stmt: "try" ":" suite except_clauses ["else" ":" suite] [finally]
        | "try" ":" suite finally   -> try_finally
finally: "finally" ":" suite
except_clauses: except_clause+
except_clause: "except" [test ["as" name]] ":" suite
// NB compile.c makes sure that the default except clause is last


with_stmt: "with" with_items ":" suite
with_items: with_item ("," with_item)*
with_item: test ["as" name]

match_stmt: "match" test ":" _NEWLINE _INDENT case+ _DEDENT

case: "case" pattern ["if" test] ":" suite

?pattern: sequence_item_pattern "," _sequence_pattern -> sequence_pattern
        | as_pattern
?as_pattern: or_pattern ("as" NAME)?
?or_pattern: closed_pattern ("|" closed_pattern)*
?closed_pattern: literal_pattern
               | NAME -> capture_pattern
               | "_" -> any_pattern
               | attr_pattern
               | "(" as_pattern ")"
               | "[" _sequence_pattern "]" -> sequence_pattern
               | "(" (sequence_item_pattern "," _sequence_pattern)? ")" -> sequence_pattern
               | "{" (mapping_item_pattern ("," mapping_item_pattern)* ","?)?"}" -> mapping_pattern
               | "{" (mapping_item_pattern ("," mapping_item_pattern)* ",")? "**" NAME ","? "}" -> mapping_star_pattern
               | class_pattern

literal_pattern: inner_literal_pattern

?inner_literal_pattern: "None" -> const_none
                      | "True" -> const_true
                      | "False" -> const_false
                      | STRING -> string
                      | number

attr_pattern: NAME ("." NAME)+ -> value

name_or_attr_pattern: NAME ("." NAME)* -> value

mapping_item_pattern: (literal_pattern|attr_pattern) ":" as_pattern

_sequence_pattern: (sequence_item_pattern ("," sequence_item_pattern)* ","?)?
?sequence_item_pattern: as_pattern
                      | "*" NAME -> star_pattern

class_pattern: name_or_attr_pattern "(" [arguments_pattern ","?] ")"
arguments_pattern: pos_arg_pattern ["," keyws_arg_pattern]
                 | keyws_arg_pattern -> no_pos_arguments

pos_arg_pattern: as_pattern ("," as_pattern)*
keyws_arg_pattern: keyw_arg_pattern ("," keyw_arg_pattern)*
keyw_arg_pattern: NAME "=" as_pattern



suite: simple_stmt | _NEWLINE _INDENT stmt+ _DEDENT

?test: or_test ("if" or_test "else" test)?
     | lambdef
     | assign_expr

assign_expr: name ":=" test

?test_nocond: or_test | lambdef_nocond

?or_test: and_test ("or" and_test)*
?and_test: not_test_ ("and" not_test_)*
?not_test_: "not" not_test_ -> not_test
         | comparison
?comparison: expr (comp_op expr)*
star_expr: "*" expr

?expr: or_expr
?or_expr: xor_expr ("|" xor_expr)*
?xor_expr: and_expr ("^" and_expr)*
?and_expr: shift_expr ("&" shift_expr)*
?shift_expr: arith_expr (_shift_op arith_expr)*
?arith_expr: term (_add_op term)*
?term: factor (_mul_op factor)*
?factor: _unary_op factor | power

!_unary_op: "+"|"-"|"~"
!_add_op: "+"|"-"
!_shift_op: "<<"|">>"
!_mul_op: "*"|"@"|"/"|"%"|"//"
!comp_op: "<"|">"|"=="|">="|"<="|"<>"|"!="|"in"|"not" "in"|"is"|"is" "not"

?power: await_expr ("**" factor)?
?await_expr: AWAIT? atom_expr
AWAIT: "await"

?atom_expr: atom_expr "(" [arguments] ")"      -> funccall
          | atom_expr "[" subscriptlist "]"  -> getitem
          | atom_expr "." name               -> getattr
          | atom

?atom: "(" yield_expr ")"
     | "(" _tuple_inner? ")" -> tuple
     | "(" comprehension{test_or_star_expr} ")" -> tuple_comprehension
     | "[" _exprlist? "]"  -> list
     | "[" comprehension{test_or_star_expr} "]"  -> list_comprehension
     | "{" _dict_exprlist? "}" -> dict
     | "{" comprehension{key_value} "}" -> dict_comprehension
     | "{" _exprlist "}" -> set
     | "{" comprehension{test} "}" -> set_comprehension
     | name -> var
     | number
     | string_concat
     | "(" test ")"
     | "..." -> ellipsis
     | "None"    -> const_none
     | "True"    -> const_true
     | "False"   -> const_false


?string_concat: string+

_tuple_inner: test_or_star_expr (("," test_or_star_expr)+ [","] | ",")

?test_or_star_expr: test
                 | star_expr

?subscriptlist: subscript
              | subscript (("," subscript)+ [","] | ",") -> subscript_tuple
?subscript: test | ([test] ":" [test] [sliceop]) -> slice
sliceop: ":" [test]
?exprlist: (expr|star_expr)
         | (expr|star_expr) (("," (expr|star_expr))+ [","]|",")
?testlist: test | testlist_tuple
testlist_tuple: test (("," test)+ [","] | ",")
_dict_exprlist: (key_value | "**" expr) ("," (key_value | "**" expr))* [","]

key_value: test ":"  test

_exprlist: test_or_star_expr (","  test_or_star_expr)* [","]

classdef: "class" name ["(" [arguments] ")"] ":" suite



arguments: argvalue ("," argvalue)*  ("," [ starargs | kwargs])?
         | starargs
         | kwargs
         | comprehension{test}

starargs: stararg ("," stararg)* ("," argvalue)* ["," kwargs]
stararg: "*" test
kwargs: "**" test ("," argvalue)*

?argvalue: test ("=" test)?


comprehension{comp_result}: comp_result comp_fors [comp_if]
comp_fors: comp_for+
comp_for: [ASYNC] "for" exprlist "in" or_test
ASYNC: "async"
?comp_if: "if" test_nocond

encoding_decl: name

yield_expr: "yield" [testlist]
          | "yield" "from" test -> yield_from

number: DEC_NUMBER | HEX_NUMBER | BIN_NUMBER | OCT_NUMBER | FLOAT_NUMBER | IMAG_NUMBER
string: STRING 


_NEWLINE: ( /\r?\n[\t ]*/ | COMMENT )+

%import common.WS_INLINE
%ignore WS_INLINE
%ignore /\\[\t \f]*\r?\n/   // LINE_CONT
%ignore COMMENT
%declare _INDENT _DEDENT



!name: NAME | "match" | "case"
NAME: /[^\W\d]\w*/
COMMENT: /\#[^\n]*/

// 1) Prefix and escape definitions (still tokens)
PREFIX       : /[uUbBfF]?[rR]?/ | /[rR][uUbBfF]/
ESCAPED_CHAR : /\\./

// 2) Short (single-line) strings as tokens
STRING_DOUBLE: "\"" ( ESCAPED_CHAR | /[^"\\]/ )* "\""
STRING_SINGLE: "'"  ( ESCAPED_CHAR | /[^'\\]/ )* "'"

// 3) Expose a parser-rule to combine them
STRING: PREFIX? (STRING_DOUBLE | STRING_SINGLE)

// 4) Triple-quote markers as tokens
TRIPLE_DQ : "\"\"\""
TRIPLE_SQ : "'''"

// 5) The “content” parts as parser-rules
long_double_content : ( ESCAPED_CHAR | "\"\"" | /[^"\\]/ )*
long_single_content : ( ESCAPED_CHAR | "''" | /[^'\\]/ )*

// 6) The long-string parser-rule
long_string : PREFIX? ( TRIPLE_DQ long_double_content TRIPLE_DQ
            | TRIPLE_SQ long_single_content TRIPLE_SQ )

_SPECIAL_DEC: "0".."9"        ("_"?  "0".."9"                       )*
BAD_DEC_NUMBER: "0" ( "_" | "0" )* "1".."9" ( "_"? "0".."9" )*
DEC_NUMBER: "0" ( "_"? "0" )*
          | "1".."9" ( "_"? "0".."9" )* 
HEX_NUMBER.2: "0" ("x" | "X") ("_"? ("0".."9" | "a".."f" | "A".."F"))+
OCT_NUMBER.2: "0" ("o" | "O") ("_"?  "0".."7"                       )+
BIN_NUMBER.2: "0" ("b" | "B") ("_"?  "0".."1"                       )+

_EXP: ("e"|"E") ["+" | "-"] _SPECIAL_DEC
DECIMAL: "." _SPECIAL_DEC | _SPECIAL_DEC "." _SPECIAL_DEC?
FLOAT_NUMBER.2: _SPECIAL_DEC _EXP | DECIMAL _EXP?
IMAG_NUMBER.2: (_SPECIAL_DEC      | FLOAT_NUMBER) ("J" | "j")

// Comma-separated list (with an optional trailing comma)
cs_list{item}: item ("," item)* ","?
_cs_list{item}: item ("," item)* ","?
"""

LONGSTRING, DEC_NUMBER, STRING

- SFT including masking (it means that LCD is needed at inference-time)
- SFT w/ "distillation" (binary cross entropy: masked probs / unmasked probs) (We get rid of LCD at inference-time)

Model : LLama 2 7B (worst model, wasn't trained on code) ; Mistral 7B (better) ; Qwen2.5/3-7B (best)

Dataset: OpenCodeInstruct (it is Python only, validated by automated unit testing and LLM-based quality judgement)

Benchmark: HumanEval, MBPP



- SMC for RL: we generate sentences using SMC (or just LCD?), and we apply RL thanks to those sentences. It might be a big problem because bad sentences are integrated into the loss, thus when we have a bad example we are moving far from those examples.

see: random down-sampling, max-reward down-sampling and max-variance down-sampling in "Not All Rollouts are Useful: Down-Sampling Rollouts
in LLM Reinforcement Learning"


The number of uncorrect sentences is much greater than the number of correct ones. Including uncorrect ones is useful as it provies information about what not to do. However, the information provided by a correct response is much more important. 
The question is, by sampling trajectories from another policy (the one defined by LCD or by SMC), can we improve the upgrading of $\theta$? To answer this, I first need to understand TRPO, PPO and GRPO mathematically (from where are the formulaes derived, the estimator of what are we computing, etc. ). I hope that by perfectly understanding this, I will be able to demonstrate that sampling trajectories from the constrained distributions can improve the estimator by diminushing the variance of the estimator?...

According to "What Makes a Reward Model a Good Teacher? An Optimization Perspective", the lower the variance of the reward $Var_{y \sim \pi_{\theta}(.|x)}(r_{RM}(x,y))$ is, the faster the maximization of the expectation $E_{x \sim \mathcal{S}}[E_{y \sim \pi_{\theta}(.|x)}[r_{RM}(x,y)]]$ can theoretically be. 

To maximize this quantity, TRPO, PPO, GRPO use an algorithm. Maybe that by sampling from a constrained distribution (locally or globally), we can improve the speed at whoch the quantity is being optimized. Or by replacing $\pi_{old}$ by $\pi_{old}^{constrained}$.



In [2]:
import outlines
import constraintlm as clm
import torch

c:\Users\cleme\anaconda3\envs\llmsmc\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from lark import Lark
from lark.indenter import PythonIndenter

kwargs = dict(postlex=PythonIndenter(), start='file_input')


python_parser = Lark(python_grammar, parser='lalr', lexer='contextual', postlex=PythonIndenter())
python_parser3 = Lark.open_from_package('lark', 'python.lark', ['grammars'], parser='lalr', **kwargs)

In [7]:
code = """def f(x):
    x = 3
    return x
"""
for tok in python_parser.lex(code):
    print(repr(tok))

Token('DEF', 'def')
Token('NAME', 'f')
Token('LPAR', '(')
Token('NAME', 'x')
Token('RPAR', ')')
Token('COLON', ':')
Token('_NEWLINE', '\n    ')
Token('_INDENT', '    ')
Token('NAME', 'x')
Token('EQUAL', '=')
Token('DEC_NUMBER', '3')
Token('_NEWLINE', '\n    ')
Token('RETURN', 'return')
Token('NAME', 'x')
Token('_NEWLINE', '\n')
Token('_DEDENT', '')


In [8]:
code = """def f(x):
    x = 3
    return x
"""

python_parser.parse(code)

Tree(Token('RULE', 'start'), [Tree(Token('RULE', 'file_input'), [Tree(Token('RULE', 'funcdef'), [Tree(Token('RULE', 'name'), [Token('NAME', 'f')]), Tree(Token('RULE', 'parameters'), [Tree(Token('RULE', 'name'), [Token('NAME', 'x')]), None, None]), None, Tree(Token('RULE', 'suite'), [Tree(Token('RULE', 'assign_stmt'), [Tree(Token('RULE', 'assign'), [Tree('var', [Tree(Token('RULE', 'name'), [Token('NAME', 'x')])]), Tree(Token('RULE', 'number'), [Token('DEC_NUMBER', '3')])])]), Tree(Token('RULE', 'return_stmt'), [Tree('var', [Tree(Token('RULE', 'name'), [Token('NAME', 'x')])])])])])])])

In [13]:
code_test = """x=10\
\ny=0 \nwhile x>0: \n    y = y + 2 \n    x = x - 1 \n"""

python_parser.parse(code_test)

Tree(Token('RULE', 'start'), [Tree(Token('RULE', 'file_input'), [Tree(Token('RULE', 'assign_stmt'), [Tree(Token('RULE', 'assign'), [Tree('var', [Tree(Token('RULE', 'name'), [Token('NAME', 'x')])]), Tree(Token('RULE', 'number'), [Token('DEC_NUMBER', '10')])])]), Tree(Token('RULE', 'assign_stmt'), [Tree(Token('RULE', 'assign'), [Tree('var', [Tree(Token('RULE', 'name'), [Token('NAME', 'y')])]), Tree(Token('RULE', 'number'), [Token('DEC_NUMBER', '0')])])]), Tree(Token('RULE', 'while_stmt'), [Tree(Token('RULE', 'comparison'), [Tree('var', [Tree(Token('RULE', 'name'), [Token('NAME', 'x')])]), Tree(Token('RULE', 'comp_op'), [Token('MORETHAN', '>')]), Tree(Token('RULE', 'number'), [Token('DEC_NUMBER', '0')])]), Tree(Token('RULE', 'suite'), [Tree(Token('RULE', 'assign_stmt'), [Tree(Token('RULE', 'assign'), [Tree('var', [Tree(Token('RULE', 'name'), [Token('NAME', 'y')])]), Tree(Token('RULE', 'arith_expr'), [Tree('var', [Tree(Token('RULE', 'name'), [Token('NAME', 'y')])]), Token('PLUS', '+'), Tre

In [10]:
model = outlines.models.transformers("Qwen/Qwen2.5-0.5B")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [11]:
generator = outlines.generate.cfg(model, python_grammar)
sequence_p = generator("In the programming language Python, a code that do : x=10, y=0, while x>0: y = y + 2, x = x - 1, would be in Python:\nx=", max_tokens=30)

print(sequence_p)

c:\Users\cleme\anaconda3\envs\llmsmc\Lib\site-packages\outlines\fsm\guide.py:110: UserWarning: Outlines' public *community-contributed* CFG structured generation is experimental. Please review https://dottxt-ai.github.io/outlines/latest/reference/generation/cfg#disclaimer
  warnings.warn(


10, y=0, while (x>0) and (y==(y+2)) and (x==(x-1


In [16]:
sequence_python = generator("You are given a list of `n` tasks, each represented as a tuple `(start, end)`, indicating the start and end times of the task. The tasks are sorted by their start times. Your goal is to determine the maximum number of non-overlapping tasks that can be selected. Two tasks are considered non-overlapping if the start time of one task is greater than or equal to the end time of the other. \
\
**Input:**\
- An integer `n` representing the number of tasks.\
- A list of `n` tuples, where each tuple `(start, end)` represents the start and end times of a task.\
\
**Output:**\
- An integer representing the maximum number of non-overlapping tasks that can be selected.\
\
**Constraints:**\
- `1 <= n <= 10^5`\
- `0 <= start < end <= 10^9`\
\
**Sample Input:**\
```\
3\
1 3\
2 5\
4 6\
```\
\
**Sample Output:**\
```\
2\
```\
```python \n", max_tokens=300)
print(sequence_python)

def max_non_overlapping_tasks(tasks):
    # Sort the tasks by their start times
    # Use a heap to keep track of the tasks that can be selected
    # The heap will store the tasks that are currently selected
    # The heap will be sorted by their end times
    # The heap will be used to select the tasks that are non-overlapping
    # The heap will be used to select the tasks that are non-overlapping
    # The heap will be used to select the tasks that are non-overlapping
    # The heap will be used to select the tasks that are non-overlapping
    # The heap will be used to select the tasks that are non-overlapping
    # The heap will be used to select the tasks that are non-overlapping
    # The heap will be used to select the tasks that are non-overlapping
    # The heap will be used to select the tasks that are non-overlapping
    # The heap will be used to select the tasks that are non-overlapping
    # The heap will be used to select the tasks that are non-overlapping
    # The he